In [1]:
import os
import glob
import pandas as pd             # pip install pandas
from tqdm import tqdm           # pip install tqdm (진행바용·선택)

# 1. 폴더 위치
FOLDER = "/content/drive/MyDrive/25년 해군 AI 경진대회/dataset/의심선박 훈련용 데이터 셋"

# 2. 모든 CSV 경로 수집 & 날짜순 정렬
csv_paths = sorted(glob.glob(os.path.join(FOLDER, "*.csv")))

# 3. 파일별 읽어서 리스트에 담기
frames = []
for fp in tqdm(csv_paths, desc="Loading CSV"):
    df = pd.read_csv(fp)                    # 필요 시 encoding='utf-8-sig' 또는 'cp949'

    # (선택) 파일명에서 날짜(YYYYMMDD) 추출해 컬럼 추가
    # ex) 20230601.csv → 2023-06-01
    date_str = os.path.splitext(os.path.basename(fp))[0]
    if date_str.isdigit() and len(date_str) == 8:
        df["date"] = pd.to_datetime(date_str, format="%Y%m%d")

    frames.append(df)

# 4. 하나로 병합
full = pd.concat(frames, ignore_index=True)

# (선택) 중복 제거: 같은 선박(MMSI) & 날짜가 겹치면 첫 번째만 유지
if {"MMSI", "date"}.issubset(full.columns):
    full = full.drop_duplicates(subset=["MMSI", "date"])

# 5. 새 CSV 저장
OUT = os.path.join(FOLDER, "combined_ais_20230601_20240531.csv")
full.to_csv(OUT, index=False, encoding="utf-8-sig")

print(f"\n✅  통합 완료: {len(csv_paths)} 개 파일 → {len(full):,} 행")
print(f"📄  저장 위치: {OUT}")


Loading CSV: 100%|██████████| 366/366 [00:38<00:00,  9.39it/s]



✅  통합 완료: 366 개 파일 → 2,678,129 행
📄  저장 위치: /content/drive/MyDrive/25년 해군 AI 경진대회/dataset/의심선박 훈련용 데이터 셋/combined_ais_20230601_20240531.csv
